**Considering 2 means tranport (bus and Flight)**

  **Finding best optimal path considering using A* Algo:**

 **1) Both cost + time**

 **2) only time**

 **3) only cost**

##Creating Node Class

In [208]:
class Node:
        
    def __init__(self, name, parent, g, h, f, transport):                                          # Initializing the class
        self.name = name
        self.parent = parent
        self.g = g                                                                      # Distance to start node
        self.h = h                                                                      # Distance to goal node
        self.f = f      
        self.transport = transport                                                            
            
   
    
    def printNode(self):                                                                # Customized Printing of nodes
        print(self.name, end = " - ")
        print(self.parent, end = " : ")
        print(self.g, end = " : ")
        print(self.h, end=" : ")
        print(self.f)

##Creating Graph Class

In [209]:
class Graph:
    
    def __init__(self, graph_dict=None, directed=True):                                 # Initialize the class
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
                
    def make_undirected(self):                                                          # Create an undirected graph by adding symmetric edges
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
                    
    def connect(self, A, B, time=1, cost = 1 ):                                                # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
        self.graph_dict.setdefault(A, {})[B] = [time, cost]
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = [time, cost]
               
          

    def getNode(self, city, heuristics, end,means_of_transport ,add=0, cost_time = 0):   
                                         # Get a specific neighbour which has minimum cost
        nodes = []
        if city  not in self.graph_dict:
          return nodes
        if(cost_time==0):
         for (b,[time, cost]) in self.graph_dict[city].items():
            nodes.append(Node(city, b, time+cost+add, heuristics[b+'-'+end][0]+heuristics[b+'-'+end][1], time+cost+heuristics[b+'-'+end][0]+heuristics[b+'-'+end][1]+add, means_of_transport ))
        
        elif(cost_time==1):
         for (b,[time, cost]) in self.graph_dict[city].items():
            nodes.append(Node(city, b, cost+add, heuristics[b+'-'+end][1], cost+heuristics[b+'-'+end][1]+add, means_of_transport ))
        else:
          for (b,[time, cost]) in self.graph_dict[city].items():
            nodes.append(Node(city, b, time+add, heuristics[b+'-'+end][0], time+heuristics[b+'-'+end][0]+add, means_of_transport ))

        return nodes

    def give_time_cost(self,city1, city2):
      return self.graph_dict[city1][city2]
   
        
    def printgraph(self):                                                               # Function to print each edge in the entire graph
         for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                print (self.graph_dict.setdefault(a,{})[b], end = " : ")
                print(a, end = " - ")
                print(b)


## function to sort frontier

In [210]:

def sort_heuristic(frontier):
  frontier = (sorted(frontier, key=lambda x: x.f))
  return frontier


## function to backtrack to get path in order

In [211]:

def backtrack(open_list, start, end):
  map = {}
  path = list()
  trans = list()
  for item in open_list:
    map[item.parent] = [item.name, item.transport]
  val = end
  while(True):
    path.append(val)
    if(val == start):
      break
    trans.append(map[val][1])
    val = map[val][0]
    
  path.reverse()
  trans.reverse()
 
  return path, trans







## A_Star function

In [212]:

def A_Star(graph1, graph2, heuristics1,heuristics2, start, end, cost_time=0):


  if((end not in graph1.graph_dict) and (end not in graph2.graph_dict)):                                                    # Incase the goal state does not exist
    print("\n\n---------------------------\nGOAL STATE DOES NOT EXIST\n---------------------------\n\n")
    return  None

  open_list = list()
  path = list()     
  trans = list()        
                                            # Will store the path we are taking
  curr_node1 = graph1.getNode(start,heuristics1, end, 'bus', 0,cost_time)  
  curr_node2 = graph2.getNode(start, heuristics2, end, 'flight', 0,cost_time)

  frontier = list()
  frontier = curr_node1 + curr_node2
  frontier = sort_heuristic(frontier)


  curr_node = frontier[0]
  visited = []
  
  visited.append(curr_node.name)

  while(True):                                                     # Runs Until we cannot find the goal state or
     

      open_list.append(curr_node)

     # path.append(curr_node.parent)
      if(curr_node.parent==end):
        path, trans = backtrack(open_list, start, end)
        break

      visited.append(curr_node.parent)
      

      if(len(frontier)>0):
        frontier.pop(0)

      curr_node1 =  graph1.getNode(curr_node.parent,heuristics1, end, 'bus',curr_node.g,cost_time)  
      curr_node2 = graph2.getNode(curr_node.parent, heuristics2, end, 'flight',curr_node.g, cost_time)

      frontier.extend(curr_node1)
      frontier.extend(curr_node2)

     
      frontier = sort_heuristic(frontier)
 

      while(len(frontier)> 0 and frontier[0].parent in visited):
        frontier.pop(0) 

      if(len(frontier)): 
        curr_node  = frontier[0]
      else:
        break


  return path, trans

## Haversine distance

In [213]:
import math

def getDistanceFromLatLonInKm(lat1,lon1,lat2,lon2):
  R = 6371 # Radius of the earth in km
  dLat = deg2rad(lat2-lat1)  # deg2rad below
  dLon = deg2rad(lon2-lon1)
  a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2) 
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  d = R * c # Distance in km
  return d


def deg2rad(deg): 
  return deg * (math.pi/180)


## defining city latitute , longitude co-ordinaates

In [214]:
lat_long = {}
lat_long['Ahmedabad'] = (23.0225,72.5714)
lat_long['Delhi'] = (28.7041,77.1025)
lat_long['Patna'] =  (25.5941,85.1376)
lat_long['Agra'] = (27.1767,78.0081)


## list of availaible bus between cities

In [215]:
bus_list  = []
bus_cost1km  = 10
bus_time1km = 5
bus_list.append(['Ahmedabad', 'Delhi'])
bus_list.append(['Ahmedabad', 'Agra'])
print(bus_list)

[['Ahmedabad', 'Delhi'], ['Ahmedabad', 'Agra']]


###Creating graph for given city with time and cost for Bus tranportation

---



In [216]:

graph1 = Graph()

In [217]:
for listt in bus_list:
  city1  = listt[0]
  city2 =  listt[1]
  lat1 = lat_long[city1][0]
  long1= lat_long[city1][1]
  lat2 = lat_long[city2][0]
  long2 = lat_long[city2][1]
  dist = getDistanceFromLatLonInKm(lat1, long1, lat2, long2)
  graph1.connect(city1, city2, (dist*bus_time1km), (dist*bus_cost1km))
  # print(dist)
  
graph1. make_undirected()

## list of availaible flights between cities

In [218]:
flight_list  = []
 
flight_cost1km  = 20
flight_time1km = 0.5
flight_list.append(['Ahmedabad', 'Patna'])
flight_list.append(['Delhi', 'Patna'])
print(flight_list)

[['Ahmedabad', 'Patna'], ['Delhi', 'Patna']]


## Creating graph for given city with time and cost for flight tranportation

In [219]:
# Create a graph -- by flight
graph2 = Graph()
    

In [220]:
for listt in flight_list:
  city1  = listt[0]
  city2 =  listt[1]
  lat1 = lat_long[city1][0]
  long1= lat_long[city1][1]
  lat2 = lat_long[city2][0]
  long2 = lat_long[city2][1]
  dist = getDistanceFromLatLonInKm(lat1, long1, lat2, long2)
  graph2.connect(city1, city2, (dist*flight_time1km), (dist*flight_cost1km))
  # print(dist)

graph2. make_undirected()


## Heuristic Data1 (for Bus [time, cost])

In [221]:

heuristics1 = {}
for city1 in lat_long:
  for city2 in lat_long:
    lat1 = lat_long[city1][0]
    long1= lat_long[city1][1]
    lat2 = lat_long[city2][0]
    long2 = lat_long[city2][1]
    dist = getDistanceFromLatLonInKm(lat1, long1, lat2, long2)
    heuristics1[city1+'-'+city2]=[(dist*bus_time1km), (dist*bus_cost1km)]

print(heuristics1)


{'Ahmedabad-Ahmedabad': [0.0, 0.0], 'Ahmedabad-Delhi': [3887.0414037348287, 7774.082807469657], 'Ahmedabad-Patna': [6522.683933386996, 13045.367866773991], 'Ahmedabad-Agra': [3580.562392045786, 7161.124784091572], 'Delhi-Ahmedabad': [3887.0414037348287, 7774.082807469657], 'Delhi-Delhi': [0.0, 0.0], 'Delhi-Patna': [4333.462681413019, 8666.925362826038], 'Delhi-Agra': [958.6235471259822, 1917.2470942519644], 'Patna-Ahmedabad': [6522.683933386996, 13045.367866773991], 'Patna-Delhi': [4333.462681413019, 8666.925362826038], 'Patna-Patna': [0.0, 0.0], 'Patna-Agra': [3657.650732408598, 7315.301464817196], 'Agra-Ahmedabad': [3580.562392045786, 7161.124784091572], 'Agra-Delhi': [958.6235471259822, 1917.2470942519644], 'Agra-Patna': [3657.650732408598, 7315.301464817196], 'Agra-Agra': [0.0, 0.0]}


### Heuristic Data2 (for Flight [time, cost])

In [222]:

heuristics2= {}
for city1 in lat_long:
  for city2 in lat_long:
    lat1 = lat_long[city1][0]
    long1= lat_long[city1][1]
    lat2 = lat_long[city2][0]
    long2 = lat_long[city2][1]
    dist = getDistanceFromLatLonInKm(lat1, long1, lat2, long2)
    heuristics2[city1+'-'+city2]=[(dist*flight_time1km), (dist*flight_cost1km)]

print(heuristics2)


{'Ahmedabad-Ahmedabad': [0.0, 0.0], 'Ahmedabad-Delhi': [388.70414037348286, 15548.165614939315], 'Ahmedabad-Patna': [652.2683933386995, 26090.735733547983], 'Ahmedabad-Agra': [358.0562392045786, 14322.249568183144], 'Delhi-Ahmedabad': [388.70414037348286, 15548.165614939315], 'Delhi-Delhi': [0.0, 0.0], 'Delhi-Patna': [433.34626814130183, 17333.850725652075], 'Delhi-Agra': [95.86235471259822, 3834.494188503929], 'Patna-Ahmedabad': [652.2683933386995, 26090.735733547983], 'Patna-Delhi': [433.34626814130183, 17333.850725652075], 'Patna-Patna': [0.0, 0.0], 'Patna-Agra': [365.7650732408598, 14630.602929634391], 'Agra-Ahmedabad': [358.0562392045786, 14322.249568183144], 'Agra-Delhi': [95.86235471259822, 3834.494188503929], 'Agra-Patna': [365.7650732408598, 14630.602929634391], 'Agra-Agra': [0.0, 0.0]}


###Calculating the throught both air and bus distances considering both efficient(cost+ time)

In [223]:


# Run search algorithm

start = 'Ahmedabad'
Goal  = 'Patna'

path, trans = A_Star(graph1, graph2 ,heuristics1,heuristics2 ,start,Goal,0)        



total_price = 0
total_time = 0
for index in range(len(path)-1):
  city1 = path[index]
  city2 = path[index+1]
  print(path[index], end='')
  if(trans[index]=='bus'):
    values = graph1.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0], ',',values[1],')','---> ', end='')
  else:
    values = graph2.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0],',',values[1] ,')','---> ', end='')

print(path[len(path)-1])

print("Total price: ",total_price)
print("Total time: ", total_time)
    


Ahmedabad --- flight ( 652.2683933386995 , 26090.735733547983 ) ---> Patna
Total price:  26090.735733547983
Total time:  652.2683933386995


###Calculating the throught both air and bus distances considering only efficient(time)

In [224]:
# Run search algorithm

#best time
start = 'Ahmedabad'
Goal  = 'Patna'

path, trans = A_Star(graph1, graph2 ,heuristics1,heuristics2 ,start,Goal, 2)        

total_price = 0
total_time = 0
for index in range(len(path)-1):
  city1 = path[index]
  city2 = path[index+1]
  print(path[index], end='')
  if(trans[index]=='bus'):
    values = graph1.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0], ',',values[1],')','---> ', end='')
  else:
    values = graph2.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0],',',values[1] ,')','---> ', end='')

print(path[len(path)-1])

print("Total price: ",total_price)
print("Total time: ", total_time)
    

Ahmedabad --- flight ( 652.2683933386995 , 26090.735733547983 ) ---> Patna
Total price:  26090.735733547983
Total time:  652.2683933386995


###Calculating the throught both air and bus distances considering only efficient (cost)

In [225]:
# Run search algorithm

start = 'Ahmedabad'
Goal  = 'Patna'

path, trans = A_Star(graph1, graph2 ,heuristics1,heuristics2 ,start,Goal, 1)        

total_price = 0
total_time = 0
for index in range(len(path)-1):
  city1 = path[index]
  city2 = path[index+1]
  print(path[index], end='')
  if(trans[index]=='bus'):
    values = graph1.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0], ',',values[1],')','---> ', end='')
  else:
    values = graph2.give_time_cost(city1, city2)
    total_time = total_time + values[0]
    total_price = total_price + values[1]
    print(' ---', trans[index],'(',values[0],',',values[1] ,')','---> ', end='')

print(path[len(path)-1])

print("Total price: ",total_price)
print("Total time: ", total_time)
    

Ahmedabad --- bus ( 3887.0414037348287 , 7774.082807469657 ) ---> Delhi --- flight ( 433.34626814130183 , 17333.850725652075 ) ---> Patna
Total price:  25107.933533121733
Total time:  4320.38767187613
